<a href="https://colab.research.google.com/github/YuzhouLin/current_proj/blob/feature-20210203-devtest/plot_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import src.utils as utils
import src.helps_pre as pre
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import optuna

In [10]:
EDL_USED = 0
# '0: no edl; 1: edl without kl; 2: edl with kl (annealing); 
# 3: edl with kl (trade-off)'
DEVICE = pre.get_device()
DATA_PATH = '/data/NinaproDB5/raw/'


def test(params):
    #  load_data
    device = torch.device('cpu')
    test_trial = params['outer_f']
    sb_n = params['sb_n']

    # Load testing Data
    inputs, targets = pre.load_data_test_cnn(
        DATA_PATH, sb_n, test_trial)

    # Load trained model
    model = utils.Model()
    model.load_state_dict(
        torch.load(params['saved_model'], map_location=device))
    model.eval()

    # Get Results
    outputs = model(inputs.to(device)).detach()

    # Load the Testing Engine
    eng = utils.EngineTest(outputs, targets)
    results = eng.get_output_results(params['acti_fun'])
    preds = results.argmax(axis=1)
    return targets, preds




In [23]:
CM_matrix = []
for EDL_used in range(4):
    params = {'edl_used': EDL_USED}

    # Get the optimal activation function
    if EDL_USED == 0:
        params['acti_fun'] = 'softmax'
    else:
        # Get from hyperparameter study
        core_path = f'study/ecnn{EDL_USED}/sb{sb_n}'
        study_path = "sqlite:///" + core_path + f"/t{test_trial}.db"
        loaded_study = optuna.load_study(
            study_name="STUDY", storage=study_path)
        temp_best_trial = loaded_study.best_trial
        params['acti_fun']  = temp_best_trial.params['evi_fun']
    total_true = np.empty([0,0])
    total_pred = np.empty([0,0])
    for test_trial in range(1, 7):
        params['outer_f'] = test_trial
        for sb_n in range(1, 11):
            params['sb_n'] = sb_n
            model_name = f"models/ecnn{EDL_USED}/sb{sb_n}_t{test_trial}.pt"
            params['saved_model'] = model_name
            y_true, y_pred = test(params)
            print(f'Testing done. sb{sb_n}-t{test_trial}')
            #ConfusionMatrixDisplay.from_predictions(y_true, y_pred)
            total_true = y_true if len(total_true) == 0 else np.append(total_true, y_true)
            total_pred = y_pred if len(total_pred) == 0 else np.append(total_pred, y_pred)

    CM_matrix.append(confusion_matrix(total_true, total_pred))
#sns.heatmap(cm_matrix, annot=True)

Number Parameters:  1191596
Testing done. sb1-t1
Number Parameters:  1191596
Testing done. sb2-t1
Number Parameters:  1191596
Testing done. sb3-t1
Number Parameters:  1191596
Testing done. sb4-t1
Number Parameters:  1191596
Testing done. sb5-t1
Number Parameters:  1191596
Testing done. sb6-t1
Number Parameters:  1191596
Testing done. sb7-t1
Number Parameters:  1191596
Testing done. sb8-t1
Number Parameters:  1191596
Testing done. sb9-t1
Number Parameters:  1191596
Testing done. sb10-t1
Number Parameters:  1191596
Testing done. sb1-t2
Number Parameters:  1191596
Testing done. sb2-t2
Number Parameters:  1191596
Testing done. sb3-t2
Number Parameters:  1191596
Testing done. sb4-t2
Number Parameters:  1191596
Testing done. sb5-t2
Number Parameters:  1191596
Testing done. sb6-t2
Number Parameters:  1191596
Testing done. sb7-t2
Number Parameters:  1191596
Testing done. sb8-t2
Number Parameters:  1191596
Testing done. sb9-t2
Number Parameters:  1191596


In [ ]:
loaded_study = optuna.load_study(study_name="STUDY", storage="sqlite:///study/ecnn/sb4/temp.db")
temp = optuna.importance.get_param_importances(loaded_study)
trial = loaded_study.best_trial
'''
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
'''

In [22]:
for a in range(4):
    print(a)

0
1
2
3


In [19]:
a = []
a.append(cm_matrix)
a.append(cm_matrix)


In [21]:
print(a[1])

[[ 61  20   9  84   8   0   2   5   0   0   0  73]
 [101 165   3  34   0   0   0   0   0   0   0   0]
 [ 32  28 260   9   0   4   7   1   0   0   0   7]
 [  5  13   0 280   0  10   2   0   0   0   0  14]
 [  0   2  22   8 251   2  41   1   0   0   0   0]
 [  0   1  13   0  22 160 104   4   0   0   0   0]
 [  0   1   0   0  53   8 316   1   0   0   0   0]
 [  0   4   0   0   2   1  72 264   0   0   0   2]
 [  0   0   0   0   0   0   0   8 179  18  24  21]
 [  8   3   5   0   0   0   1   0   2 167  29  28]
 [  3   0   0   0   0   0   0   0 138   0 121  20]
 [  0   0   0   0   0   0   0   0  39   0  26 198]]


In [ ]:
hyperparams_ecnn = ['optimizer', 'lr', 'batch_size', 'KL', 'evi_fun']
h_ecnn_i = {k:[] for k in hyperparams_ecnn}
best_h_ecnn = {k:[] for k in hyperparams_ecnn}
print(h_ecnn_i)

hyperparams_cnn = ['optimizer', 'lr', 'batch_size']
h_cnn_i = {k:[] for k in hyperparams_cnn}
best_h_cnn = {k:[] for k in hyperparams_cnn}
print(h_cnn_i)

In [ ]:
m = ['cnn','ecnn']
for i in m:
    path = 'sqlite:///study/'+ i
    for sb_n in range(1,10):
        temp_path = path + '/sb' +str(sb_n) + '/temp.db'    
        loaded_study = optuna.load_study(study_name="STUDY", storage=temp_path)
        temp_i = optuna.importance.get_param_importances(loaded_study)
        temp_best_trial = loaded_study.best_trial
        if i == 'cnn':
            for key, value in temp_best_trial.params.items():
                h_cnn_i[key].append(temp_i[key])
                best_h_cnn[key].append(value)
        else:
            for key, value in temp_best_trial.params.items():
                h_ecnn_i[key].append(temp_i[key])
                best_h_ecnn[key].append(value)

In [ ]:
best_h_cnn

In [ ]:
###  Calculate Vacuity and dissonance 

def Bal(a,b):
    if a*b !=0:
        output = 1-abs(a-b)/(a+b)
    else:
        output = 0
    return output        

#evidence = np.array([99]*12)
evidence = np.array([4,9,0])
alpha = evidence + 1
prob_SL = alpha / np.sum(alpha)
class_num = len(evidence)
total_evidence = np.sum(alpha)

u_va = class_num / total_evidence

print('The vacuity is: %.2f' %(u_va))

belief = evidence/total_evidence
belief = belief.tolist()
u_dis = 0.0
for index_k,k in enumerate(belief):
    temp0 = 0.0
    temp1 = 0.0
    for index_j,j in enumerate(belief):
        if index_j!=index_k:
            temp0 += j*Bal(k,j)
            temp1 += j
    if temp1!=0:
        u_dis += k*temp0/temp1
print('The dissonance is: %.2f' %(u_dis))

In [ ]:
import torch
#alpha = torch.ones([1, 12], dtype=torch.float32)
belief = torch.tensor([[0.3,0.3,0.0],[0.1,0.1,0.1]], dtype=torch.float32,requires_grad=True)
k_class = 3

'''
def Bal(a,b,batch_n):
    output = torch.where(a*b>0,1.0-torch.abs(a-b)/(a+b),0)
    if a*b !=0:
        output = 1.0-torch.abs(a-b)/(a+b)
    else:
        output = torch.Tensor.zeros(batch_n,1)
    return output
'''

batch_n = belief.size()[0]
u_dis = torch.zeros(batch_n,1)
for index_k in range(k_class):
    temp0 = torch.zeros(batch_n,1)
    temp1 = torch.zeros(batch_n,1)
    for index_j in range(k_class):
        if index_j!=index_k:
            k = belief[:,index_k].reshape(batch_n ,1)
            j = belief[:,index_j].reshape(batch_n ,1)
            temp0 += j*(1.0-torch.abs(k-j)/(k+j+1e-8))
            #torch.where(k*j>0.0,1.0-torch.abs(k-j)/(k+j),torch.zeros(batch_n,1))
            temp1 += j
    u_dis += k*temp0/(temp1+1e-8)

u_dis.sum().backward()
print(belief.grad.data)

In [ ]:
import numpy as np
a = np.array([[1.0],[2.0],[3.0]])
b = np.array([[2.0],[1.0],[10.0]])
c = np.concatenate((a,b),axis=1)
print(np.shape(a))
d = np.max(c,axis=1,keepdims=True)
print(np.shape(d))
print(d)

In [ ]:
import torch
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]
 
w = torch.Tensor([1.0]) # w的初值为1.0
w.requires_grad = True # 需要计算梯度


#print(w.grad.data.type())



def forward(x):
    return x*w  # w是一个Tensor
 
 
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y)**2
 
print("predict (before training)", 4, forward(4).item())
 
for epoch in range(10):
    for x, y in zip(x_data, y_data):
        l =loss(x,y) # l是一个张量，tensor主要是在建立计算图 forward, compute the loss
        l.backward() #  backward,compute grad for Tensor whose requires_grad set to True

        print(w.data.type())
        print(w.grad.data.type())
        break
        print('\tgrad:', x, y, w.grad.item())
        w.data = w.data - 0.01 * w.grad.data   # 权重更新时，需要用到标量，注意grad也是一个tensor
 
        w.grad.data.zero_() # after update, remember set the grad to zero
 
    print('progress:', epoch, l.item()) # 取出loss使用l.item，不要直接使用l（l是tensor会构建计算图）
 
print("predict (after training)", 4, forward(4).item())


In [ ]:
evidence = np.array([19]*12)
print(evidence)

In [ ]:
### Loss function
def loss_cal(para):
    y_label = np.array([1,0,0,0,0,0,0,0,0,0,0,0])
    
    # parameters: alphas
    S = np.sum(para)
    pred = para/S
    pred_err = np.sum(np.square(pred-y_label))

    pred_var = np.sum(pred*(1-pred)/(S+1))
    #pred_var = np.sum(pred*(1-pred))
    return pred_err,pred_var


In [ ]:
def kl_divergence(para,n_class=3):
    beta = np.ones()


In [ ]:
alphas = np.array([10,12,10,10,10,10,10,10,10,10,10,10])
pred_err,pred_var = loss_cal(alphas)
print(pred_err,pred_var,pred_err+pred_var)

In [ ]:
prob = np.array([0,0.5,0])
print(mse_loss(prob))
prob = np.array([0,0.4,0.3])
print(mse_loss(prob))

In [ ]:
alphas = np.array([1,10,8])
alphas = (alphas-1)*(1-np.array([1,0,0]))+1
print(alphas)
beta = np.ones(3)
#beta = np.array([8,8,8])
S_alpha = np.sum(alphas)
print(S_alpha)
S_beta = np.sum(beta)
print(S_beta)
lnB = sc.gammaln(S_alpha) - np.sum(sc.gammaln(alphas))
print(lnB)
lnB_uni = np.sum(sc.gammaln(beta)) - sc.gammaln(S_beta)
print(lnB_uni)
dg0 = sc.digamma(S_alpha)
dg1 = sc.digamma(alphas)
kl = np.sum((alphas-beta)*(dg1-dg0))+lnB+lnB_uni
print(kl)

In [ ]:
y = np.array([1,0,0])
kl_alpha = (alphas - 1) * (1 - y) + 1
print(kl_alpha)

In [ ]:
import torch
#alpha = torch.ones([1, 12], dtype=torch.float32)
alpha = torch.tensor([[1,20,1,1,18,1,1,18,1,1,1,1]], dtype=torch.float32)
beta = torch.ones([1, 12], dtype=torch.float32)
S_alpha = torch.sum(alpha, dim=1, keepdim=True)
print(S_alpha)
S_beta = torch.sum(beta, dim=1, keepdim=True)
print(S_beta)
lnB = torch.lgamma(S_alpha) - \
    torch.sum(torch.lgamma(alpha), dim=1, keepdim=True)
print(lnB)
lnB_uni = torch.sum(torch.lgamma(beta), dim=1,
                    keepdim=True) - torch.lgamma(S_beta)
print(lnB_uni)
dg0 = torch.digamma(S_alpha)
dg1 = torch.digamma(alpha)

kl = torch.sum((alpha - beta) * (dg1 - dg0), dim=1,
                keepdim=True) + lnB + lnB_uni

print(kl)

In [ ]:
y = torch.tensor([[1,0,0,0,0,0,0,0,0,0,0,0]], dtype=torch.float32)
#print(1-torch.sum(alpha*y,dim=1,keepdim=True)/torch.sum(alpha, dim=1, keepdim=True))

In [ ]:
y2 = torch.tensor([[2,3,0,0,0,0,0,0,0,0,0,0]], dtype=torch.float32)
#print(torch.dot(y2,y))

In [ ]:
y3 = torch.tensor([[2],[3],[4]])
print(y2.reshape(3,4)/y3)

In [ ]:
value = 5
temp = y[y!=torch.max(y)]
print(temp)

In [ ]:
a = [1,2,3]
a[0],a[1],a[2] = a[2],a[1],a[0]
print(a)


In [ ]:
import torch
import matplotlib.pyplot as   
a = torch.linspace(0, 1, 10, requires_grad=True)
b = torch.tensor([0.5], requires_grad=True)
loss = torch.abs(a-b)/(a+b+1e-8)
loss.backward() 
plt.plot(a.detach().numpy(), a.grad.detach().numpy(), label='grad')
plt.legend()

In [ ]:
import torch
a = torch.tensor([0.3],requires_grad=True)
b = torch.tensor([0.2],requires_grad=True)
#loss = torch.where(a*b>0,torch.abs(a-b)/(a+b+1e-8), torch.abs(a-b)/(a+b+1e-8))
#loss = torch.where(a*b>0,torch.abs(a-b)/(a+b+1e-8), torch.zeros(1,1))
loss = 1-torch.abs(a-b)/(a+b+1e-8)
#loss = torch.where(torch.abs(a-b)<=1e-3, 0.5*(a-b)**2/(a+b+1e-8), (1e-3*torch.abs(a-b)-0.5*1e-3**2)/(a+b+1e-8))
loss.backward()
print(a.grad.data)

In [ ]:
a = torch.tensor([[10.,2.0,3.0]],requires_grad=True)
b = torch.sum(a,dim=1, keepdim=True)
loss = torch.log(b)
loss.backward()
print(a.grad.data)

In [ ]:
a = torch.tensor([0.4],requires_grad=True)
loss = (a-0.5)/(a+0.5)
loss.backward()
print(a.grad.data)

In [ ]:
index = y>0
y2 = torch.tensor([[2,0.5,0,0,0,0,0,0,0,0,0,0]])
print(index*y+y2*(~index))

In [ ]:
outputs = torch.tensor([[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0,0],[1,0.9,0.8,0.7,0.6,5.5,0.4,0.3,0.2,0.1,0,0]], dtype=torch.float32)
print(outputs.size())
_, preds = torch.max(outputs, 1)
print(preds)

print(torch.eq(preds, preds))

mean = torch
var =  torch.tensor([[0.1,0.3]], dtype=torch.float32)
index = var > 0.5
print(index)
index = mean + var > 0.5
print(index)
print(mean + var)
_, preds = torch.max(outputs, 1)

In [ ]:
y = torch.tensor([[1,0,0],[1,0,0],[1,0,0]], dtype=torch.float32)

In [ ]:
def get_device():
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    return device

def loglikelihood_loss(y, alpha, device=None):
    if not device:
        device = get_device()
    y = y.to(device)
    alpha = alpha.to(device)
    S = torch.sum(alpha, dim=1, keepdim=True)
    loglikelihood_err = torch.sum(
        (y - (alpha / S)) ** 2, dim=1, keepdim=True)
    loglikelihood_var = torch.sum(
        alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)
    #loglikelihood = loglikelihood_err + loglikelihood_var
    return loglikelihood_err,loglikelihood_var

def kl_divergence(alpha, num_classes, device=None):
    if not device:
        device = get_device()
    beta = torch.ones([1, num_classes], dtype=torch.float32, device=device)
    S_alpha = torch.sum(alpha, dim=1, keepdim=True)
    S_beta = torch.sum(beta, dim=1, keepdim=True)
    lnB = torch.lgamma(S_alpha) - \
        torch.sum(torch.lgamma(alpha), dim=1, keepdim=True)
    lnB_uni = torch.sum(torch.lgamma(beta), dim=1,
                        keepdim=True) - torch.lgamma(S_beta)

    dg0 = torch.digamma(S_alpha)
    dg1 = torch.digamma(alpha)

    kl = torch.sum((alpha - beta) * (dg1 - dg0), dim=1,
                   keepdim=True) + lnB + lnB_uni
    return kl

In [ ]:
import torch.nn.functional as F
y = torch.tensor([[1,0,-1],[0,1,0],[0,0,1]]*3, dtype=torch.float32)
print(F.softmax(y,dim=1))


In [ ]:
y2 = torch.tensor([1,2,3,4,5,6,7,8,9]).reshape(9,1)
print(y*y2)


In [ ]:
temp = [[3,1,1], # ideal
        [1,2,1], # ideal
        [1,1,3], # ideal
        [10,8,1], # practial
        [8,10,1], # practical 
        [1,2,3], # practical
        [1,4,1], # wrong 
        [3,1,1], # wrong 
        [1,2,1]  # wrong 
        ]


In [ ]:
alpha = torch.tensor(temp, dtype=torch.float32)
#A_err,A_var = loglikelihood_loss(y, alpha)
kl_alpha = (alpha - 1) * (1 - y) + 1
kl_div = kl_divergence(kl_alpha, 3)
#loss = torch.mean(A+kl_div)


In [ ]:
print(A_err)
print(kl_div)
print(kl_div+A_err)
print(torch.mean(A_err+kl_div))

In [ ]:
y = torch.tensor([[1.0,2.0],[2.0,0.5],[0.2,3]], dtype=torch.float32)
label = torch.tensor([1,0,0])
_,p = torch.max(y,1)
print(torch.where(label==p,1,-1))

In [ ]:
y[:,-1].reshape(3,1).size()
#print(a)

In [ ]:
y[:,:-1]